# Efficacy of Vocational Rehabilitation Services in America 

> Dataset: [Current Population Survey, July 2021: Disability Supplement](https://api.census.gov/data/2021/cps/disability/jul.html)
* The universe consists of all persons in the civilian non-institutional population of the United States living in households. 
* The probability sample selected to represent the universe consists of approximately 50,000 households.

### Imports

In [8]:
import requests
import pandas as pd

### Query Texas Records

In [23]:
HOST = "https://api.census.gov/data"
year = "2021"
dataset = "cps/disability/jul"
base_url = "/".join([HOST, year, dataset])

predicates = {}
get_vars = ["PEMLR", "PESD6A", "PESD6B", "PESD6C", "PESD6D", "PESD6E", "PESD6F", "PESD6G", "PESD7A", "PESD7B", "PESD7C","PESD7E", "PESD7G"]
predicates["get"] = ",".join(get_vars)
predicates["for"] = "county:*"
predicates["in"] = "state:48"

r = requests.get(base_url, params=predicates)

[["PEMLR","PESD6A","PESD6B","PESD6C","PESD6D","PESD6E","PESD6F","PESD6G","PESD7A","PESD7B","PESD7C","PESD7E","PESD7G","state","county"],
["5","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","139"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","139"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","139"],
["1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["7","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","-1","48","0"],
["6","-1","-1",

### Format Data Frame

In [27]:
col_names = ["Labor Force-employment status", "Used Vocational Rehabilitation Agencies", "Used One Stop Career Centers?", "Used the Ticket to Work Program?", "Used Assistive Technology Act Prog?", "Used Ctr for Indept Living for Ind w/ Dis?", "Used The Client Assistance Program?", "Used any other employment assistance program?", "How helpful was Vocational Rehab Agency?", "How helpful One Stop Career Centers?", "The Ticket to Work Program helpfulness?", "Ctr for Indpdt Living for Ind w/ Dis helpful", "Other employment assist program helpful?", "state", "county"]
df = pd.DataFrame(columns=col_names, data=r.json()[1:])

print(df.head(100))

   Labor Force-employment status Used Vocational Rehabilitation Agencies  \
0                              5                                      -1   
1                             -1                                      -1   
2                             -1                                      -1   
3                              1                                      -1   
4                              1                                      -1   
..                           ...                                     ...   
95                             5                                      -1   
96                             6                                       2   
97                             1                                      -1   
98                             1                                      -1   
99                             1                                      -1   

   Used One Stop Career Centers? Used the Ticket to Work Program?  \
0                 